In [96]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import matplotlib.pyplot as plt
import json

ASSUME_TZ = None
HORIZON_H = 6
LOOKBACK_L = 48
TEST_SAMPLES = 1000

FEATURES = ['price_pct_change', 'volume_pct_change']

In [97]:
model = tf.keras.models.load_model("./models/model.keras")

In [98]:
eth_data = pd.read_csv('./paper_wallet_data/eth_data')
eth_data = eth_data.dropna().reset_index(drop=True)
eth_data.head()

,Unnamed: 0,time,ETH_price,ETH_volume
0,12063,2016-06-09 15:00:00,14.69,333.920307
1,12064,2016-06-09 16:00:00,14.69,667.084306
2,12065,2016-06-09 17:00:00,14.57,355.767139
3,12066,2016-06-09 18:00:00,14.65,561.489527
4,12090,2016-06-10 18:00:00,14.10,1238.313305


In [99]:
eth_data['time'] = pd.to_datetime(eth_data['time'], utc=True)
if ASSUME_TZ is not None:
    eth_data['time'] = eth_data['time'].dt.tz_convert('UTC')

eth_data["hour"] = eth_data["time"].dt.hour

In [100]:
eth_data['price_pct_change'] = eth_data['ETH_price'].pct_change(1)   
eth_data['volume_pct_change'] = eth_data['ETH_volume'].pct_change(1)
eth_data = eth_data.dropna().reset_index(drop=True)
hours = eth_data['hour']
hours = np.array(hours, dtype=np.int32)   # force integers 0–23

y = eth_data['ETH_price'].pct_change(HORIZON_H)

In [101]:
#get ETH id
with open("paper_wallet_data/asset_to_id.json", "r") as f:
    asset_to_id = json.load(f)

# Example usage
eth_id = asset_to_id["ETH"]
print(f"ETH Id: {eth_id}")

ETH Id: 8


In [102]:
def make_sequences(df, L):
    X_list = []
    X = df[FEATURES].values
    for t in range(L, len(df)):
            X_list.append(X[t-L:t,:])
    return np.stack(X_list, dtype=np.float64)

X = make_sequences(eth_data, LOOKBACK_L)

In [103]:
A = np.full((TEST_SAMPLES,), eth_id, dtype=np.int32)

In [104]:
def make_recent_windows(features, seq_len=LOOKBACK_L, n_samples=TEST_SAMPLES, asset_id=0):
    N = features.shape[0]
    X_list = []
    for t in range(N - seq_len - n_samples, N - seq_len):
        if X_list and X_list[0].shape == features[t:t+seq_len].shape:
            X_list.append(features[t:t+seq_len])
        elif not X_list:
            X_list.append(features[t:t+seq_len])
    
    X = np.stack(X_list[:n_samples])  # (n_samples, seq_len, feat_dim)
    A = np.full((n_samples,), asset_id, dtype=np.int32)  # (n_samples,)
    return X, A

In [105]:
features = eth_data[FEATURES].values[:TEST_SAMPLES]
X, A = make_recent_windows(features, seq_len=LOOKBACK_L, n_samples=TEST_SAMPLES, asset_id=eth_data)

ValueError: could not broadcast input array from shape (66864,7) into shape (1000,)

In [94]:
A.shape

(1000,)

In [95]:
yhat = model.predict({"seq": X, "asset_id": A})
print(yhat.shape)  # (1000, 1)

AttributeError: 'str' object has no attribute 'shape'

In [190]:
def compute_true_targets(prices, horizon=3, n_samples=TEST_SAMPLES):
    """
    Compute true forward % change targets for the last n_samples windows.
    
    prices: array-like of raw prices
    horizon: prediction horizon (steps ahead)
    n_samples: how many sequences we predicted
    seq_len: length of each input window (48 in your case)
    """
    prices = np.asarray(prices, dtype=np.float32)
    
    # forward return for every valid index
    forward = (prices[horizon:] - prices[:-horizon]) / prices[:-horizon]
    
    # slice the last n_samples, aligned with your X windows
    true_vals = forward[-n_samples:]
    
    return true_vals

true_changes = compute_true_targets(prices, horizon=3, n_samples=1000)

/var/folders/1f/5lhk7sb97rz53v7gk1vkcjkr0000gn/T/ipykernel_11255/64026007.py:13: RuntimeWarning: divide by zero encountered in divide
  forward = (prices[horizon:] - prices[:-horizon]) / prices[:-horizon]
/var/folders/1f/5lhk7sb97rz53v7gk1vkcjkr0000gn/T/ipykernel_11255/64026007.py:13: RuntimeWarning: invalid value encountered in divide
  forward = (prices[horizon:] - prices[:-horizon]) / prices[:-horizon]


In [191]:
# Metrics
mae = np.mean(np.abs(yhat - true_changes))
#corr = np.corrcoef(yhat, true_changes)[0,1]
directional_acc = np.mean(np.sign(yhat) == np.sign(true_changes))

print(f"MAE: {mae:.4f}, DirAcc: {directional_acc:.2%}")

MAE: inf, DirAcc: 45.45%
